In [1]:
from dotenv import load_dotenv
import pytest
from web3 import Web3
import web3
from multicall.call import Call, NOT_A_CONTRACT_REVERT_MESSAGE
import os

load_dotenv()

BLOCK_TO_CHECK = 18_000_000
w3 = Web3(Web3.HTTPProvider(os.environ.get("ALCHEMY_URL")))
w3

In [2]:
highest_finalized_block = w3.eth.get_block(block_identifier="finalized", full_transactions=False)["number"]
# only cache things that have been finalized
highest_finalized_block

19825616

In [3]:
import sqlite3
from multicall.call import Call

CACHE_PATH = "cache_db.sqlite"

if os.path.exists(CACHE_PATH):
    os.remove(CACHE_PATH)

conn = sqlite3.connect(CACHE_PATH)

cursor = conn.cursor()

# Create the multicallCache table
# Ensure callId is a primary key for quick lookups
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS multicallCache (
    callId BLOB PRIMARY KEY,
    target TEXT,
    signature TEXT,
    argumentsAsStr TEXT,
    argumentsAsPickle bytes,
    block INTEGER,
    chainId INTEGER,   
    success BOOLEAN,
    response BLOB
)
"""
)

conn.commit()

cursor.execute("SELECT * FROM multicallCache WHERE callId = ?", ("exampleCallId",))
result = cursor.fetchone()
conn.close()

In [4]:
conn = sqlite3.connect(CACHE_PATH)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
conn.close()
tables

[('multicallCache',)]